<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/pyro_introduction_custom_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyro

## An object-based toolbox for robot dynamic simulation, analysis, control and planning


This page is an introduction to the basic functionnality of pyro, an open-source python library developped at Université de Sherbrooke and hosted here: https://github.com/SherbyRobotics/pyro

## Tutorial content


1.   [The Dynamic System class and basic functionnality](https://colab.research.google.com/drive/18eEL-n-dv9JZz732nFCMtqMThDcfD2Pr?usp=sharing)
2.   **Creating a custom dynamic class (this page)**
3.   [Closed-loop system and controllers objects](https://colab.research.google.com/drive/1mog1HAFN2NFEdw6sPudzW2OaTk_li0Vx?usp=sharing)
4.   The Linear System class (comin soon..)
4.   The Mechanical System class (coming soon..)
5.   [The Manipulator Robot class](https://colab.research.google.com/drive/1OILAhXRxM1r5PEB1BWaYtbR147Ff3gr1?usp=sharing)


## Importing Pyro

This code show a quick exemple of

1.   Cloning pyro code source from the github repository
2.   Adding pyro folder to the python path
3.   Importing the library to the python interpreter

In [ ]:
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro


Here we import other basic python tools:

*   Numpy: the python library for linear algebra, on top of which pyro is built. 
*   Display: that is needed to show animation in the colab environment. If pyro is used locally then this is not needed.
*   Inspect: that we will use here only for printing source code in for this tutorial

In [ ]:
import numpy as np
from IPython import display
import inspect

# The Dynamic system class

Pyro is built on the concept of dynamic system classes, with standarized methods and properties. Multiple analysis and control tools are built to use the standarize methodes and properties.

**The core of a dynamic system is defined by a differential equation:** \\
$\dot{x} = f(x,u,t)$ \\
where $x$ is a state vector, $u$ is a control input vector $u$ and $t$ is the time. This function caracterize the dynamics of the system, and is used to compute the evolution of the system when a simulation is conducted.

Three (optionnal) functions can be defined, to take advantage of additionnal tools:
* **The output equation** $y = h(x,u,t)$, that compute variables that sensor would read for usage by a controller (by default $y = x$)
* **The configuration equation** $q = xu2q(x,u,t)$, that compute variables describing the configuration of the system (by default $q = x$)
* **The forward kinematic equation** $lines = forward\_kinematic\_line( q )$, that compute a graphic output for given configurations variable $q$ (by default the graphical outputs are dots with coordinates (x=q,y=0,z=0).

The following figure summarize how those function are related using a block diagram representation. 
<img width="900" src="https://user-images.githubusercontent.com/16725496/163312300-faa7fe2c-178e-4c58-ae6c-4b256fd9ab92.jpg" class="center">

# A simple custom class

Here we create a simple custom dynamic system class. We will represent the dynamics:

$ \dot{x} = x + u + t$

All custom class should inherite from another dynamic system class, the most generic mother class to use is the system.ContinuousDynamicSystem that include methods generic to all types of dynamic systems.

In [ ]:
from pyro.dynamic import system


##############################################################################
        
class MyCustomClass( system.ContinuousDynamicSystem ):

    
    ############################
    def __init__(self):
        """ """

        # Dimensions of signal --> Many method will acces those attributes
        self.n = 1   
        self.m = 1   
        self.p = 1
        
        # This initialize standard params
        system.ContinuousDynamicSystem.__init__( self, self.n, self.m, self.p)
        
        # Here we shows how to customize some labels

        self.name = 'The name of my system'

        self.state_label = ['The name of the state variable']
        self.input_label = ['The name of the input variable']
        self.output_label = ['The name of the output variable']

        self.state_units = ['[]']
        self.input_units = ['[]']
        self.output_units = ['[]']


    #############################
    def f(self, x, u, t):

        dx = u + x + t  # State derivative vector, this is the differential equation defining the system
        
        return dx

This is enough to use it!

We can simulate the system and use analysis tools:

In [ ]:
sys = MyCustomClass()

sys.x0   = np.array([ 0 ])
sys.ubar = np.array([ 0 ])

sys.compute_trajectory( tf = 3 )

sys.plot_trajectory('xu')

By default the graphic output is just dots, one for each state variable:

In [ ]:
# This would work locally in a python console
#sys.animate_simulation()

# This is the way for showing an animation on colab (we need to generate html)
ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

We can customize the graphic animation by modifying the forward_kinematic_lines method:

In [ ]:
def graph( q ):

  lines_pts   = [] # list of array (n_pts x 3) for each lines
  lines_style = []
  lines_color = []
  
  # dot no 1
  pts      = np.zeros(( 1 , 3 ))
  pts[0,:] = np.array([ q[0], 0, 0])
  
  lines_pts.append( pts )
  lines_style.append( 'o')
  lines_color.append( 'b' )
  
  # dot no 2
  pts      = np.zeros(( 1 , 3 ))
  pts[0,:] = np.array([ 0, q[0], 0])
  
  lines_pts.append( pts )
  lines_style.append( 'x')
  lines_color.append( 'r' )
  
  # dot no 3
  pts      = np.zeros(( 1 , 3 ))
  pts[0,:] = np.array([ 0, 0, q[0]])
  
  lines_pts.append( pts )
  lines_style.append( 'o')
  lines_color.append( 'k' )
          
  return lines_pts , lines_style , lines_color

# Overloading the graphic function
sys.forward_kinematic_lines = graph

Lets re-compute the animation:

In [ ]:
# This would work locally in a python console
#sys.animate_simulation()

# This is the way for showing an animation on colab (we need to generate html)
ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

Note that all animation can be shown in 3D with this argument:

In [ ]:
# This would work locally in a python console
#sys.animate_simulation( is_3d = True )

# This is the way for showing an animation on colab (we need to generate html)
ani  = sys.generate_simulation_html_video( is_3d = True )
html = display.HTML( ani )
display.display(html)